In [ ]:
### TO DO
## ITEMS:
# Ores (DONE)
# Components (DONE)
#- Grab the standard recipe for each component
# Equipment
## BUILDINGS (DONE)
## RECIPES
#- Obtain alternate recipes from Hard Drive page

In [1]:
import requests
import bs4
base_url = 'https://satisfactory.fandom.com/wiki/{}'
import numpy as np
import pandas as pd
import re

In [2]:
### This is the code to grab the lists of items

##Gets the list of ore names
ores = requests.get('https://satisfactory.fandom.com/wiki/Category:Ores')
soup = bs4.BeautifulSoup(ores.text,"lxml")
oretext = soup.select('div.mw-category-group')
ore_list = set()

txt = oretext[0]

for item in oretext:
    new = item.find_all('a')
    for sub in new:
        ore_list.add(sub['title'])
            
oredered_list = list(ore_list)

In [47]:

def get_comp_dict():
    '''Ouputs a list of components and their corresponding data in DataFrame format'''
    #Gets the list of components and conveniently the tier data as part of a filter 
    # because I'm bad at scraping.
    #Ouputs a dictionary of Component Name:Tier
    #Start by getting the ores witht their tiers.
    item_page = requests.get('https://satisfactory.fandom.com/wiki/Satisfactory_Wiki')
    soup = bs4.BeautifulSoup(item_page.text,"lxml")
    componentvomit = soup.select('.navbox')[14]
    example = componentvomit.find_all('a', title=True)
    comp_list = {}
    comp_dict = {}
    cat = ""
    for item in example:
        if 'Tier' in item['title'] or 'MAM' in item['title']:
            cat = item['title']
            cat.replace('Tier ','')
        else:
            comp_list[item['title']] = cat
    comp_list['Biomass'] = 'Tier 0'
    comp_list.pop('Research',None)
    
    comp_keys = [*comp_list]
    
    ##Extract relevant values from item pages
    tier_list = []
    sink_value_list = []
    recipe_list_list = []
    
    for key in comp_keys:
        
        page = requests.get(base_url.format(key))
        soup = bs4.BeautifulSoup(page.text,"lxml")
        
        #Research Tier
        # MAM recipes don't have tiers
        if comp_list[key] == "MAM":
            infobox = soup.select('.portable-infobox')
            a_s = infobox[0].find_all('a')[1]
            try:
                tier = a_s['title']
                tier = tier.replace("Tier ", "")
                #maybe remove this replace function and add it in when the list is called.
            except:
                tier = "Not Implemented"
        else:    
            tier = comp_list[key]
            
        #Sink Value
        try:
            string = infobox[2].text
            sink_value = re.sub(r'\n','',string)
        except:
            sink_value = "NaN"
        #Alternate Recipe Names
        try:
            craft = soup.select('.wikitable')[0]
            rows = craft.find_all('tr')
        except:
            craft = []

        alt_recipes = []
        for ind in range(8):
            try:
                if "Alternate" in rows[ind+2].find('td').text:
                    alt = rows[ind+2].find('td').text
                    alt.replace("Alternate","")
                    altalt = re.sub(r'Alternate','',alt)
                    altalt = re.sub(r": ", "", altalt)
                    alt_recipes.append(altalt)
            except:
                pass
        
        #Append values to lists
        tier_list.append(tier)
        sink_value_list.append(sink_value)
        recipe_list_list.append(alt_recipes)
    
    comp_dict["Tier"] = tier_list
    comp_dict["Sink Value"] = sink_value_list
    comp_dict["Alternate Recipes"] = recipe_list_list
    #Last step - return the names of items and comp_dict in the same order
    return comp_keys, comp_dict
        

In [48]:
comp_dict = get_comp_dict()
ComponentTable = pd.DataFrame(comp_dict, index=com)
ComponentTable.head()



IndexError: list index out of range

In [49]:
item_page = requests.get('https://satisfactory.fandom.com/wiki/Satisfactory_Wiki')
soup = bs4.BeautifulSoup(item_page.text,"lxml")
componentvomit = soup.select('.navbox')[14]
example = componentvomit.find_all('a', title=True)
comp_list = {}
comp_dict = {}
cat = ""
for item in example:
    if 'Tier' in item['title'] or 'MAM' in item['title']:
        cat = item['title']
        cat.replace('Tier ','')
    else:
        comp_list[item['title']] = cat
comp_list['Biomass'] = 'Tier 0'
comp_list.pop('Research',None)

comp_keys = [*comp_list]

for key in comp_keys:
        
    page = requests.get(base_url.format(key))
    soup = bs4.BeautifulSoup(page.text,"lxml")
    

    #Research Tier
    try:
        craft = soup.select('.wikitable')[0]
        rows = craft.find_all('tr')
    except:
        craft = []
    break

In [83]:
len(comp_keys)

88

In [79]:
  
page = requests.get(base_url.format('Iron Ingot'))
soup = bs4.BeautifulSoup(page.text,"lxml")


#Research Tier
try:
    craft = soup.select('.wikitable')[0]
    rows = craft.find_all('tr')
except:
    craft = []
craft

<table class="wikitable" style="text-align:center"><tbody><tr><th>Recipe</th><th colspan="12">Ingredients</th><th>Building</th><th colspan="2">Products</th><th>Prerequisites</th></tr><tr class="firstRow" style="border-top: 2px solid #575757"><td rowspan="1" style="line-height:1; position:relative; overflow:hidden;">Iron Ingot</td><td colspan="12" rowspan="1" style="border:0; line-height:1; padding:0.5em"><div style="line-height:1"><div style="font-size:medium; display:inline-table"><span style="cursor: help; border-bottom: none;" title="The amount of this resource per crafting cycle, from which the per minute value is calculated">1 × </span><a href="/wiki/Iron_Ore" title="Iron Ore"><img alt="Iron Ore.png" data-file-height="256" data-file-width="256" data-image-key="Iron_Ore.png" data-image-name="Iron Ore.png" decoding="async" height="40" src="https://static.wikia.nocookie.net/satisfactory_gamepedia_en/images/8/87/Iron_Ore.png/revision/latest/scale-to-width-down/40?cb=20200602231757" wi

In [82]:
rows[1]

<tr class="firstRow" style="border-top: 2px solid #575757"><td rowspan="1" style="line-height:1; position:relative; overflow:hidden;">Iron Ingot</td><td colspan="12" rowspan="1" style="border:0; line-height:1; padding:0.5em"><div style="line-height:1"><div style="font-size:medium; display:inline-table"><span style="cursor: help; border-bottom: none;" title="The amount of this resource per crafting cycle, from which the per minute value is calculated">1 × </span><a href="/wiki/Iron_Ore" title="Iron Ore"><img alt="Iron Ore.png" data-file-height="256" data-file-width="256" data-image-key="Iron_Ore.png" data-image-name="Iron Ore.png" decoding="async" height="40" src="https://static.wikia.nocookie.net/satisfactory_gamepedia_en/images/8/87/Iron_Ore.png/revision/latest/scale-to-width-down/40?cb=20200602231757" width="40"/></a><br/><div style="font-size:x-small; text-align:right">Iron Ore</div></div></div><div style="font-weight:bold; line-height:2"><span style="cursor: help; border-bottom: no

In [76]:
comp_keys[4]

'Iron Ingot'

In [77]:
requests.get('https://satisfactory.fandom.com/wiki/Iron Ingot')

<Response [200]>

In [201]:
test = requests.get(base_url.format('Cable'))
soup = bs4.BeautifulSoup(test.text,"lxml")
craft = soup.select('.wikitable')[0]
rows = craft.find_all('tr')

alt_recipes = []
for ind in range(8):
    try:
        if "Alternate" in rows[ind+2].find('td').text:
            alt = rows[ind+2].find('td').text
            alt.replace("Alternate","")
            altalt = re.sub(r'Alternate','',alt)
            alt_recipes.append(altalt)
    except:
        pass
    
print(base_recipe)
alt_recipes
#rows = craft.find_all('tr')
#rows

Cable


['Coated Cable', 'Insulated Cable', 'Quickwire Cable']

In [192]:
"Alternate" in trying.text

True

In [94]:
#Grab important values for each ore
#Tier, Sink Value, Mining Speed, No. of nodes 
def get_ore_dict():
    ore_dict = {}
    tier_list = []
    sink_value_list = []
    mining_list = []
    impure_list = []
    normal_list = []
    pure_list = []
    
    for ore in oredered_list:
        
        name = requests.get(base_url.format(ore))
        soup = bs4.BeautifulSoup(name.text,"lxml")

        #Tier Acquired
        infobox = soup.select('.infobox-row-value')
        a_s = infobox[0].find('a')
        try:
            tier = a_s['title']
            tier = tier.replace("Tier ", "")
            #maybe remove this replace function and add it in when the list is called.
        except:
            tier = "Not Implemented"

        #Sink Value
        try:
            string = infobox[2].text
            sink_value = re.sub(r'\n','',string)
        except:
            sink_value = "NaN"

        #Mining Speed - Mk.1 - Impure
        mining = soup.select('.tabbertab')[0]
        mining = mining.find('td').text


        #No. of nodes - Returns dict with node types and values
        nodes = soup.select('.wikitable')[2]
        node_types = nodes.find_all('td')
        node_names = ["Impure","Normal","Pure"]
        node_amounts = {}
        for node in node_names:
            next_num = False
            for item in node_types:
                node_found = False
                #If the node type was found in the previous loop, gets the quantity.
                while next_num:
                    node_amounts[current] = item.text
                    next_num = False
                #Checks if the node type was found so the next loop it'll look for the number    
                if node in item:
                    node_found = True
                    while node_found:
                        current = node
                        next_num = True
                        node_found = False

        #Append new values to list for dictionary
        tier_list.append(tier)
        sink_value_list.append(sink_value)
        mining_list.append(mining)
        impure_list.append(node_amounts['Impure'])
        normal_list.append(node_amounts['Normal'])
        pure_list.append(node_amounts['Pure'])
        
    ore_dict["Tier"] = tier_list
    ore_dict["Sink Value"] = sink_value_list
    ore_dict["Mining Speed"] = mining_list
    ore_dict["Impure Nodes"] = impure_list
    ore_dict["Normal Nodes"] = normal_list
    ore_dict["Pure Nodes"] = pure_list
        
    return ore_dict
    
    #Final section will take all previous results and append them all to a DataFrame

In [96]:
ore_dict = get_ore_dict()
OreTable = pd.DataFrame(ore_dict, index=oredered_list)
OreTable.head()

,Tier,Sink Value,Mining Speed,Impure Nodes,Normal Nodes,Pure Nodes
Uranium,8,35,30,1,3,0
Copper Ore,0,3,30,9,28,12
Raw Quartz,Quartz Research,15,30,0,11,5
Caterium Ore,Caterium Research,7,30,0,8,8
Iron Ore,Onboarding,1,30,33,41,46


In [237]:
### This is the code to grab the list of buildings
builres = requests.get("https://satisfactory.fandom.com/wiki/Buildings")
soup = bs4.BeautifulSoup(builres.text,"lxml")
#This is the relevant information for buildings: name, ingredients.
btable = soup.select('.wikitable')[0]

try1 = btable.select('tr')
building_list = {}

#Grabs all 171 building names
for item in try1:
    
    tds = item.select('td')
    
    try:
        bs = tds[2].select('b')
        a_s = tds[2].select('a')
        
        #Grabs building names
        Building = tds[0].text
           
        
        #Grabs ingredient quantities
        blist = []
        for item in bs:
            blist.append(item.text)
        blist

        #Grabs ingredient names
        adict = {}
        for item in a_s:
            adict[item['title']]=""
        #Combines all into a dict of buildings: dict of ingredients to quantities
        building_list[Building] = dict(zip(adict, blist))
    except:
        pass
building_list    
#You combine the names and quantities of ingredients with
#dict_name = dict(zip(adict, blist))


{'Assembler': {'Reinforced Iron Plate': '8x', 'Rotor': '4x', 'Cable': '10x'},
 'AWESOME Shop': {'Screw': '200x', 'Iron Plate': '10x', 'Cable': '30x'},
 'AWESOME Sink': {'Reinforced Iron Plate': '15x',
  'Cable': '30x',
  'Concrete': '45x'},
 'Biomass Burner': {'Iron Plate': '15x', 'Iron Rod': '15x', 'Wire': '25x'},
 'Blender': {'Motor': '20x',
  'Heavy Modular Frame': '10x',
  'Aluminum Casing': '50x',
  'Radio Control Unit': '5x'},
 'Ceiling Light': {'Quartz Crystal': '20x', 'Wire': '16x', 'Steel Beam': '6x'},
 'Center Door Wall (Plating)': {'Iron Plate': '3x', 'Concrete': '3x'},
 'Center Door Wall (Sheet Metal)': {'Iron Plate': '3x', 'Concrete': '3x'},
 'Coal Generator': {'Reinforced Iron Plate': '20x',
  'Rotor': '10x',
  'Cable': '30x'},
 'Constructor': {'Reinforced Iron Plate': '2x', 'Cable': '8x'},
 'Conveyor Belt Mk.1': {'Iron Plate': '1x'},
 'Conveyor Belt Mk.2': {'Reinforced Iron Plate': '1x'},
 'Conveyor Belt Mk.3': {'Steel Beam': '1x'},
 'Conveyor Belt Mk.4': {'Encased Indus

In [ ]:
### This is the code to grab the list of recipes
#Because of the way recipes are stored in the wiki, 
#you have to get the basic recipe from each item page
#Then you can get the list of alternate recipes from the hard drive page

##Relevant data to get for each recipe:
#Recipe Name,Product, Building, Ingredients, Rate/min, Prerequisites

In [214]:
alt_recipes_list = ComponentTable["Alternate Recipes"].tolist()
alt_recipes_set = set()
alt_recipes_dict = {}
for item in alt_recipes_list:
    for sub in item:
        alt_recipes_set.add(sub)
        alt_recipes_dict[sub] = "Ingredients"


In [219]:
alternates = requests.get('https://satisfactory.fandom.com/wiki/Hard_Drive')
soup = bs4.BeautifulSoup(alternates.text,'lxml')
test = soup.select('.wikitable')[1]

In [258]:
test = soup.select('.wikitable')[1]
another = test.find_all('tr')[8]
idict = {}
key = ""
#for item in another:
onemore = another.find_all('a')
key = onemore[0].text
another.find_all('td')
#code is smth like for each item in test.find_all('tr')
#tr[1] is the name
#tr[2] is the product
#tr[3] contains ingredients
#tr[4] is the rate/min
#tr[7] is the prerequisites

[<td style="text-align:left;position:relative; overflow:hidden;padding-top:2.2em;padding-bottom:2.2em;"><a class="mw-redirect" href="/wiki/Nobelisk#Alternate_recipes_analysis" title="Nobelisk">Seismic Nobelisk</a></td>,
 <td><a class="image" href="/wiki/File:Nobelisk.png"><img alt="Nobelisk.png" data-file-height="256" data-file-width="256" decoding="async" height="30" src="https://static.wikia.nocookie.net/satisfactory_gamepedia_en/images/7/76/Nobelisk.png/revision/latest/scale-to-width-down/30?cb=20190430195815" width="30"/></a> <a class="mw-redirect" href="/wiki/Nobelisk" title="Nobelisk">Nobelisk</a></td>,
 <td style="vertical-align:right;"><a class="image" href="/wiki/File:Black_Powder.png"><img alt="Black Powder.png" data-file-height="256" data-file-width="256" decoding="async" height="30" src="https://static.wikia.nocookie.net/satisfactory_gamepedia_en/images/3/3e/Black_Powder.png/revision/latest/scale-to-width-down/30?cb=20190430200121" width="30"/></a> <a href="/wiki/Black_Powd